In [ ]:
from evo.tools import plot
from evo.tools import  file_interface
from evo.core import sync
from evo.core import metrics
import evo.main_ape as main_ape
import evo.common_ape_rpe as common
import pprint
from evo.tools import log
log.configure_logging(verbose=True, debug=True, silent=False)

import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
%matplotlib inline


# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False


dataset="MH_04_difficult"
# dataset="MH_03_medium"
loop_closure=False
output_exp_data="./exp_data/results_"+dataset+str(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))+".txt"

if loop_closure:
    with open("./vins_result_loop.csv","r") as raw_result_file ,open(output_exp_data,'w') as output_file:
        for line in  raw_result_file:
            data=line.split(",")
            time_stamp=float(data[0])/1e9
            output_file.write(" ".join([str(time_stamp)]+data[1:-1])+'\n')
else:
    with open("./vins_result_no_loop.csv","r") as raw_result_file ,open(output_exp_data,'w') as output_file:
        for line in  raw_result_file:
            data=line.split(",")
            time_stamp=float(data[0])/1e9
            output_file.write(" ".join([str(time_stamp)]+data[1:-4])+'\n')
traj_ref = file_interface.read_euroc_csv_trajectory("./config/"+dataset+"/data.csv")
traj_est = file_interface.read_tum_trajectory_file(output_exp_data)

print(traj_ref)
print(traj_est)
traj_ref_0=traj_ref
traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est)

In [ ]:
import copy

traj_est_aligned = copy.deepcopy(traj_est)
traj_est_aligned.align(traj_ref, correct_scale=True, correct_only_scale=False)

fig = plt.figure()
traj_by_label = {
    "estimate (not aligned)": traj_est,
    "estimate (aligned)": traj_est_aligned,
    "reference": traj_ref,
}
plot.trajectories(fig, traj_by_label, plot.PlotMode.xyz)
plt.show()

pose_relation = metrics.PoseRelation.translation_part
use_aligned_trajectories = True

if use_aligned_trajectories:
    data = (traj_ref, traj_est_aligned) 
else:
    data = (traj_ref, traj_est)

ape_metric = metrics.APE(pose_relation)
ape_metric.process_data(data)

ape_stat = ape_metric.get_statistic(metrics.StatisticsType.rmse)
print(ape_stat)
ape_stats = ape_metric.get_all_statistics()
pprint.pprint(ape_stats)

In [ ]:
seconds_from_start = [t - traj_est.timestamps[0] for t in traj_est.timestamps]
fig = plt.figure()

plot.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                 statistics={s+f" {v:.4f}":v for s,v in ape_stats.items() if s != "sse"},
                 name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
plt.show()

In [ ]:
plot_mode = plot.PlotMode.xy
fig = plt.figure()
ax = plot.prepare_axis(fig, plot_mode)
plot.traj(ax, plot_mode, traj_ref, '--', "gray", "reference")
plot.traj_colormap(ax, traj_est_aligned if use_aligned_trajectories else traj_est, ape_metric.error, 
                   plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
ax.legend()
plt.show()